In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from PyPDF2 import PdfReader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.llms.base import LLM
from pydantic import PrivateAttr

e:\KULIAH\SMT 6\Fintune model langachain\venv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_text_from_pdfs(folder_path):
    text = ""
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            try:
                reader = PdfReader(os.path.join(folder_path, file))
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
            except Exception as e:
                print(f"Gagal memproses {file}: {e}")
    return text

In [3]:
def build_faiss_index(text, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    if not text.strip():
        return None
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_text(text)
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return FAISS.from_texts(chunks, embeddings)

In [4]:
def load_faiss_retriever(path, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    if not os.path.exists(path):
        return None
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    index = FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)
    return index.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [5]:
class CustomLLM(LLM):
    _model: any = PrivateAttr()
    _tokenizer: any = PrivateAttr()
    _device: str = PrivateAttr(default="cuda" if torch.cuda.is_available() else "cpu")

    def __init__(self, model, tokenizer, device=None, **kwargs):
        super().__init__(**kwargs)
        self._model = model
        self._tokenizer = tokenizer
        self._device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    @property
    def _llm_type(self):
        return "custom-mistral-lora"

    def _call(self, prompt: str, stop=None) -> str:
        inputs = self._tokenizer(prompt, return_tensors="pt").to(self._device)
        if "token_type_ids" in inputs:
            del inputs["token_type_ids"]
        outputs = self._model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.9,       # tingkatkan supaya output lebih variatif
            top_p=0.95,            # sampling yang lebih luas
            do_sample=True,
            pad_token_id=self._tokenizer.eos_token_id
        )
        response = self._tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Hapus prompt dari hasil output agar hanya jawaban yang keluar
        return response[len(prompt):].strip()


In [6]:
def combined_retriever(query, retrievers):
    docs = []
    for retriever in retrievers:
        if retriever:
            docs += retriever.get_relevant_documents(query)
    return docs

In [7]:
def classify_food_or_drink(query: str) -> str:
    """Klasifikasi sederhana untuk membedakan makanan atau minuman."""
    minuman_keywords = ["susu", "teh", "kopi", "air", "jus", "sirup", "soda", "minuman", "milk", "drink"]
    makanan_keywords = ["nasi", "roti", "daging", "telur", "sayur", "makanan", "makan", "ayam", "ikan", "mie", "rice", "food", "bread"]

    query_lower = query.lower()
    if any(word in query_lower for word in minuman_keywords):
        return "minuman"
    elif any(word in query_lower for word in makanan_keywords):
        return "makanan"
    else:
        return "tidak diketahui"



In [8]:
def answer_query(query, llm, retrievers):
    jenis = classify_food_or_drink(query)

    docs = combined_retriever(query, retrievers)

    if docs:
        context = "\n\n---\n\n".join(
            [f"Sumber: {d.metadata.get('source', 'unknown')}\nKonten: {d.page_content}" for d in docs]
        )
        prompt = f"""
Anda adalah asisten nutrisi yang sangat membantu dan informatif.

Jenis bahan dalam pertanyaan ini adalah: {jenis}.

Jawab pertanyaan berdasarkan konteks berikut:

{context}

Pertanyaan: {query}

Jawaban:
"""
    else:
        prompt = f"""
Anda adalah asisten nutrisi yang sangat membantu dan informatif.

Jenis bahan dalam pertanyaan ini adalah: {jenis}.

Jawab pertanyaan berikut berdasarkan pengetahuan umum Anda.

Pertanyaan: {query}

Jawaban:
"""

    print("Prompt yang dikirim ke model:\n", prompt)  # Debug
    answer = llm._call(prompt)
    return answer, docs


In [9]:
# ===== Konfigurasi Path dan Model =====
pdf_folder = "WHO_doc"  # Folder berisi PDF WHO/FAO
index_path = "faiss_who_index"
adapter_path = "./mistral-lora-adapter"  # Path ke adapter LoRA
base_model_name = "mistralai/Mistral-7B-v0.1"

# ===== Cek dan Bangun FAISS Index jika belum ada =====
if not os.path.exists(index_path):
    print("Membangun indeks FAISS dari dokumen PDF...")
    raw_text = extract_text_from_pdfs(pdf_folder)
    index = build_faiss_index(raw_text)
    if index:
        index.save_local(index_path)
        print("Indeks FAISS berhasil dibuat dan disimpan.")
    else:
        print("Gagal membangun indeks FAISS.")
else:
    print("Menggunakan indeks FAISS lokal yang sudah ada.")

# ===== Load Tokenizer dan Model =====
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    torch_dtype=torch.float16
)
model.eval()

llm = CustomLLM(model, tokenizer)

# ===== Load FAISS Retriever =====
retriever = load_faiss_retriever(index_path)
retrievers = [retriever]

Menggunakan indeks FAISS lokal yang sudah ada.


Loading checkpoint shards: 100%|██████████| 2/2 [00:27<00:00, 13.53s/it]
C:\Users\Farhan Rahmansyah\AppData\Local\Temp\ipykernel_11072\3852927642.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [11]:
# ===== Fungsi utama untuk menjawab pertanyaan =====
def main():
    question = "Berapa kandungan nutrisi dalam air?"
    print(f"\nPertanyaan: {question}\n")

    answer, docs = answer_query(question, llm, retrievers)
    print("💬 Jawaban:\n", answer)


if __name__ == "__main__":
    main()


Pertanyaan: Berapa kandungan nutrisi dalam air?

Prompt yang dikirim ke model:
 
Anda adalah asisten nutrisi yang sangat membantu dan informatif.

Jenis bahan dalam pertanyaan ini adalah: minuman.

Jawab pertanyaan berdasarkan konteks berikut:

Sumber: unknown
Konten: (https://extranet.who.int/nutrition/gina/en/node/41857, accessed 18 January 2023).
101. Government of Finland. Peraturan Menteri Kesehatan Nomor 30 tentang pencantuman informasi andijngan gula, garam, 
dan lemak serta pesan kesehatan untuk pangan olahan dan pangan siap saji [Inclusion of sugar, salt and fat contents and 
health message on processed and fast foods] 2013 (https://extranet.who.int/nutrition/gina/en/node/26167).
102. Peraturan Menteri Kesehatan Nomor 30 tentang pencantuman informasi andijngan gula, garam, dan lemak serta pesan 
kesehatan untuk pangan olahan dan pangan siap saji [Inclusion of sugar, salt and fat contents and health message 
on processed and fast foods]. Jakarta: Government of Indonesia; 2017 